In [1]:
# Imports
import pandas as pd
import time
import aiohttp 
import asyncio
import socket
import ssl
import time

In [2]:
original_dataset_path = './malicious_phish.csv'
new_dataset_path = 'filtered_html_enhanced.csv'
html_file_folder_path = './files'

data = pd.read_csv(original_dataset_path)
data.head()

                                                      url        type  \
0                                        br-icloud.com.br    phishing   
1                     mp3raid.com/music/krizz_kaliko.html      benign   
2                         bopsecrets.org/rexroth/cr/1.htm      benign   
3       http://www.garage-pirenne.be/index.php?option=...  defacement   
4       http://adventure-nicaragua.net/index.php?optio...  defacement   
...                                                   ...         ...   
651186            xbox360.ign.com/objects/850/850402.html    phishing   
651187       games.teamxbox.com/xbox-360/1860/Dead-Space/    phishing   
651188         www.gamespot.com/xbox360/action/deadspace/    phishing   
651189      en.wikipedia.org/wiki/Dead_Space_(video_game)    phishing   
651190          www.angelfire.com/goth/devilmaycrytonite/    phishing   

                   error  status_code content_type  \
0       connection_reset            0          NaN   
1              

C:\Users\Alex\AppData\Local\Temp\ipykernel_20080\637519168.py:1: DtypeWarning: Columns (2,4,5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('./malicious_html_enhanced.csv')


In [5]:

headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
        # Add other headers as needed
}

# Sends an HTTP requests with multiple retries
async def get_response(session, url,retries=5):
    data = {
        "success" : False,
        "content" : None
    }
    for x in range(retries):
        try:
            async with session.get(url,headers=headers,timeout=10,allow_redirects=True) as resp:
                data["success"] = True
                data["response"] = resp
                try:
                    data["content"] = await resp.read()
                    data["string_content"] = await resp.text()
                    return data
                except aiohttp.ClientPayloadError as e:
                    if x == retries-1:
                        data["success"] = False
                        data["error"] = "server_disconnected"
                        data["except_error"] = e 
                        return data
                    else:
                        continue
                
        except aiohttp.ClientConnectorCertificateError as e:
            data["error"] = "certificate"
            data["except_error"] = e 
            return data
        except aiohttp.ClientConnectorError as e:
            data["success"] = False
            data["content"] = None
            data["except_error"] = e 
            if isinstance(e.os_error, socket.gaierror):
                #print("DNS resolution error:", e)
                data["error"] = "dns"
                # Handle DNS resolution errors (e.g., hostname not found)
            elif isinstance(e.os_error, ConnectionResetError):
                #print("Connection Reset error",e)
                data["error"] = "connection_reset"
            elif isinstance(e.os_error, ConnectionRefusedError):
                data["error"] = "connection_refused"
            elif isinstance(e.os_error, ssl.SSLError):
                data["error"] = "ssl"
                #print("SSL Handshake Error",e)
            else:
                print("Other connection error:", e.args)
                data["error"] = "unknown"
            return data
            
        except aiohttp.ClientOSError as e:
            #print(f"Network error occurred: {e}")
            #print("Retrying...")
            await asyncio.sleep(1)  # Wait for a short duration before retrying
            if x == retries-1:
                data["error"] = "os"
                data["except_error"] = e 
                return data
        except aiohttp.ServerDisconnectedError as e:
            #print(f"Server disconnected: {e}")
            #print("Retrying...")
            await asyncio.sleep(1)
            if x == retries-1:
                data["error"] = "server_disconnected"
                data["except_error"] = e 
                return data
        except asyncio.TimeoutError:
            data["error"] = "timeout"
            return data
        except aiohttp.ClientResponseError as e:
            data["error"] = "client"
            data["except_error"] = e
            #print(e)
            return data
        except UnicodeError as e:
            data["error"] = "unicode"
            return data
        except ConnectionResetError as e:
            data["error"] = "connection_reset"
            return data
        except:
            data["error"] = "unknown"
            return data
    	

# Proccesses URLS in batches to prevent overloading netcode
async def proccess_link_batch(urls):
    start_time = time.time()
    async with aiohttp.ClientSession() as session:
        tasks = [asyncio.create_task(get_response(session, url)) for url in urls]
        results = await asyncio.gather(*tasks)
        print(f"{(time.time() - start_time):.2f} seconds")
        return results

# Saves Results to csv file
def proccess_results(results,indices):
    for result,index in zip(results,indices):

        if result["success"]:
            response = result["response"]
            data.at[index,'error'] = "none"
            data.at[index,"status_code"] = response.status
            data.at[index,"content_type"] = response.content_type
            data.at[index,"headers"] = dict(response.headers.items())
            data.at[index,"num_headers"] = len(data.at[index,"headers"])
            data.at[index,"cookies"] = dict(response.cookies.items())
            data.at[index,"num_cookies"] = len(data.at[index,"cookies"])

            if response.history:
                data.at[index,"num_redirects"] = len(response.history)
                                  
            if result["content"] != None:
                # Write content to file
                path = f"{html_file_folder_path}/{index}.html"
                with open(path, 'wb') as f:
                    f.write(result["content"])
                    
                data.at[index,"content_length"] = len(result["content"])
        else:
            data.at[index,'error'] = result['error']
            

def initialize_pandas_columns(columns):
    for name,default in columns:
        if name not in data.columns:
            data[name] = default


new_columns = [("error","n/a"), ("status_code",0), ("content_type", None), ("headers", None),
               ("num_headers", 0), ("cookies", None), ("num_cookies", 0), ("num_redirects", 0), ("content_length",0)]
initialize_pandas_columns(new_columns)

urls = []
indices = []
for index, row in data.iterrows():
    url = row['url'] 
    if not (url.startswith('http://') or url.startswith('https://')):
        url = 'https://' + url
        
    urls.append(url)
    indices.append(index)
    if index % 500 == 499:
        results = await proccess_link_batch(urls)
        proccess_results(results, indices)

        urls = []
        indices = []

    if index % 2000 == 0:
        print(index)
        try:
            data.to_csv(new_dataset_path, mode='w', index=False)
        except:
            print(sys.exc_info()[1])
            pass
        


10.76 seconds
10.73 seconds
10.64 seconds
10.73 seconds
102000
11.07 seconds
10.72 seconds
10.71 seconds
10.71 seconds
104000
10.09 seconds
10.71 seconds
10.72 seconds


Exception in callback _ProactorBasePipeTransport._call_connection_lost(None)
handle: <Handle _ProactorBasePipeTransport._call_connection_lost(None)>
Traceback (most recent call last):
  File "C:\Users\Alex\anaconda3\Lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\Alex\anaconda3\Lib\asyncio\proactor_events.py", line 165, in _call_connection_lost
    self._sock.shutdown(socket.SHUT_RDWR)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback _ProactorBasePipeTransport._call_connection_lost(None)
handle: <Handle _ProactorBasePipeTransport._call_connection_lost(None)>
Traceback (most recent call last):
  File "C:\Users\Alex\anaconda3\Lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\Alex\anaconda3\Lib\asyncio\proactor_events.py", line 165, in _call_connection_lost
    self._sock.shutdown(socket.SHUT_RDWR)

10.72 seconds
106000
10.07 seconds
10.72 seconds
10.73 seconds
10.68 seconds
108000


Exception in callback _ProactorBasePipeTransport._call_connection_lost(None)
handle: <Handle _ProactorBasePipeTransport._call_connection_lost(None)>
Traceback (most recent call last):
  File "C:\Users\Alex\anaconda3\Lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\Alex\anaconda3\Lib\asyncio\proactor_events.py", line 165, in _call_connection_lost
    self._sock.shutdown(socket.SHUT_RDWR)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host


10.06 seconds
10.71 seconds
10.71 seconds


...

Exception in callback _ProactorBasePipeTransport._call_connection_lost(None)
handle: <Handle _ProactorBasePipeTransport._call_connection_lost(None)>
Traceback (most recent call last):
  File "C:\Users\Alex\anaconda3\Lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\Alex\anaconda3\Lib\asyncio\proactor_events.py", line 165, in _call_connection_lost
    self._sock.shutdown(socket.SHUT_RDWR)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback _ProactorBasePipeTransport._call_connection_lost(None)
handle: <Handle _ProactorBasePipeTransport._call_connection_lost(None)>
Traceback (most recent call last):
  File "C:\Users\Alex\anaconda3\Lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\Alex\anaconda3\Lib\asyncio\proactor_events.py", line 165, in _call_connection_lost
    self._sock.shutdown(socket.SHUT_RDWR)

10.67 seconds


Can not load response cookies: Illegal key '&?+nCDBF2LhANhGaZMhwkzPM'


10.52 seconds
10.74 seconds
10.69 seconds
216000
10.61 seconds
Other connection error: (ConnectionKey(host='charlescounty.org', port=443, is_ssl=True, ssl=True, proxy=None, proxy_auth=None, proxy_headers_hash=None), OSError(22, 'The semaphore timeout period has expired', None, 121, None))
10.71 seconds
10.71 seconds
10.71 seconds
218000
10.45 seconds
10.69 seconds


ValueError: URL should be absolute